# Clustering graph into k definite subsets
<img src="image.png">
 


In [1]:
import numpy as np
# Graph class

class Node:
    def __init__(self, id):
        self.id = id

    def get_id(self):
        return self.id
    
class Edge:
    def __init__(self, source, dest):
        self.source = source
        self.dest = dest

class Graph:

    def __init__(self, num_of_nodes):
        self.num_of_nodes = num_of_nodes
        self.nodes = []
        self.edges = []
        for i in range(num_of_nodes):
            self.nodes.append(Node(i))
        self.adjacent_list = [[] for i in range(num_of_nodes)]
        self.adjacent_list_transpose = [[] for i in range(num_of_nodes)]

    def get_node(self, id):
        """
        :param id:
        :return: The object Node with the given id
        """
        return self.nodes[id]

    def add_edge(self, edge):
        self.edges.append(edge)
        self.adjacent_list[edge.source.id].append(edge.dest)
        self.adjacent_list_transpose[edge.dest.id].append(edge.source)

    def get_adjacent_nodes(self, node):
        return self.adjacent_list[node.id]

    def get_adjacent_nodes_transpose(self, node):
        return self.adjacent_list_transpose[node.id]
    
    def get_D_matrix(self):
        # sum row-wise elements of Adjacency matrix
        D = np.zeros((len(self.nodes), len(self.nodes)))
        
        for i in range(len(self.nodes)):
            D[i][i] = len(self.get_adjacent_nodes(Node(i)))
            
        return D
    
    def get_A_matrix(self):
        A = np.zeros((len(self.nodes), len(self.nodes)))
        
        for node in self.nodes:
            for neigh in self.get_adjacent_nodes(node):
                A[node.id][neigh.id] = 1
            
        return A


In [2]:
# Graph Reader 

class GraphReader():
    def __init__(self, path, is_undirected=False):
        self.path = path
        self.is_undirected = is_undirected
        self.dict_inv = {}

    def read_graph(self):
        """
        Dataset is in txt each line is in the format "id_src, id_dst"
        :return:
        """
        filename = self.path
        edges = []
        with open(filename) as f:
            for line in f:
                edges.append([int(n) for n in line.strip().split(",")])

        nodes = []
        for e in edges:
            nodes.append(e[0])
            nodes.append(e[1])

        nodes = list(set(nodes))
        
        # Create mapping with ids from 0 to len(set(nodes)) -1
        new_id = 0
        dict_nodes_id = {}
        for el in list(set(nodes)):
            dict_nodes_id[el] = new_id
            self.dict_inv[new_id] = el
            new_id+=1

        myGraph = Graph(len(nodes))
        for el in edges:#[:-1]:
            curr_edge = Edge(Node(dict_nodes_id[el[0]]), Node(dict_nodes_id[el[1]]))
            myGraph.add_edge(curr_edge)

            if self.is_undirected:
                curr_edge_inv = Edge(Node(dict_nodes_id[el[1]]), Node(dict_nodes_id[el[0]]))
                myGraph.add_edge(curr_edge_inv)

        return myGraph



In [3]:
path = "input.txt"
graph_reader = GraphReader(path, True)
graph = graph_reader.read_graph()

In [4]:
import numpy as np
from sklearn.preprocessing import normalize

# Defining parameters
noise = 1
A = graph.get_A_matrix()
D = graph.get_D_matrix()
X = normalize(A, norm='l1')

In [5]:
print(A)
print(D)

[[0. 1. 1. 0. 0. 0. 1. 1. 0.]
 [1. 0. 1. 0. 0. 1. 1. 1. 0.]
 [1. 1. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 1.]
 [0. 0. 0. 1. 0. 1. 0. 0. 1.]
 [0. 1. 0. 1. 1. 0. 0. 0. 1.]
 [1. 1. 1. 0. 0. 0. 0. 1. 0.]
 [1. 1. 1. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0.]]
[[4. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 5. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 4. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 3. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 3. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 4. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 4. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 4. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 3.]]


In [6]:
from numpy.linalg import matrix_power
D_1_2 = D.copy()
for i in range(len(D)):
    D_1_2[i][i] = D_1_2[i][i]**(-0.5)
print(D_1_2)


[[0.5        0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.4472136  0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.5        0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.57735027 0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.57735027 0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.5
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.5        0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.5        0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.57735027]]


In [7]:
# Our laplacian as D^-0.5 dot A dot D^-0.5
L_temp = np.dot(D_1_2, A)
L = np.dot(L_temp, D_1_2)
print(L)

[[0.         0.2236068  0.25       0.         0.         0.
  0.25       0.25       0.        ]
 [0.2236068  0.         0.2236068  0.         0.         0.2236068
  0.2236068  0.2236068  0.        ]
 [0.25       0.2236068  0.         0.         0.         0.
  0.25       0.25       0.        ]
 [0.         0.         0.         0.         0.33333333 0.28867513
  0.         0.         0.33333333]
 [0.         0.         0.         0.33333333 0.         0.28867513
  0.         0.         0.33333333]
 [0.         0.2236068  0.         0.28867513 0.28867513 0.
  0.         0.         0.28867513]
 [0.25       0.2236068  0.25       0.         0.         0.
  0.         0.25       0.        ]
 [0.25       0.2236068  0.25       0.         0.         0.
  0.25       0.         0.        ]
 [0.         0.         0.         0.33333333 0.33333333 0.28867513
  0.         0.         0.        ]]


In [8]:
# Eigenpair
from numpy import linalg as LA
eigenval, eigenvect = LA.eig(L)
print(eigenval)

[ 1.          0.90664812 -0.06486199 -0.42511946 -0.25       -0.25
 -0.33333333 -0.25       -0.33333333]


In [39]:
# get matrix X of largest k eigenvalues' eigenvectors
k = 3
X = eigenvect[:,:k]
Y = X#normalize(X, norm='l2')
print(X)

[[ 0.34299717  0.2927103 ]
 [ 0.38348249  0.20505869]
 [ 0.34299717  0.2927103 ]
 [ 0.29704426 -0.40826537]
 [ 0.29704426 -0.40826537]
 [ 0.34299717 -0.33939922]
 [ 0.34299717  0.2927103 ]
 [ 0.34299717  0.2927103 ]
 [ 0.29704426 -0.40826537]]


In [40]:
# K-means
from sklearn.cluster import KMeans
import numpy as np

n_clusters = k
kmeans = KMeans(n_clusters=n_clusters , random_state=0).fit(Y)
kmeans.labels_

array([1, 1, 1, 0, 0, 0, 1, 1, 0], dtype=int32)

In [41]:
nodes_with_clusters = kmeans.predict(Y)

In [42]:
clusters = [[] for i in range(n_clusters )]
for i in range(len(nodes_with_clusters)):
    print(graph_reader.dict_inv[i])
    clusters[nodes_with_clusters[i]].append(graph_reader.dict_inv[i])
print(clusters)

1
2
3
4
5
6
11
12
13
[[4, 5, 6, 13], [1, 2, 3, 11, 12]]
